In [2]:
# imports
import os
from typing import Dict, List
from dotenv import load_dotenv
from IPython.display import display, Markdown
from litellm import completion

In [3]:
# Load environment variables
load_dotenv(override=True)

# Keys (optional: LiteLLM reads from env automatically if set)
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
# Prefer GEMINI_API_KEY for LiteLLM's Gemini provider
gemini_api_key = os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_API_KEY")

In [4]:
# Models
models: Dict[str, str] = {
    "gpt":    "gpt-4o-mini",
    "claude": "anthropic/claude-3-haiku-20240307",
    "gemini": "gemini/gemini-2.0-flash",
}

# System prompts
prompts: Dict[str, str] = {
    "Athena": (
        "You are Athena, a systems thinker. "
        "Map assumptions, constraints, and risks. "
        "Be concise; use numbered points. "
        "End with one clarifying question. "
        "Keep responses short (1–2 sentences), but specific."
    ),
    "Blaze": (
        "You are Blaze, a pragmatic engineer. "
        "Propose implementable steps, quick tests, and simple metrics. "
        "Give costs/effort at a high level. "
        "End with a concrete next action. "
        "Keep responses short (1–2 sentences), but insightful."
    ),
    "Cypher": (
        "You are Cypher, a critical evaluator. "
        "Challenge claims and surface uncertainties. "
        "Suggest minimal validation experiments or data checks. "
        "Avoid repeating others verbatim. "
        "Keep responses short (1–2 sentences), but imaginative."
    ),
}


In [5]:
def chat_with_agent(
    role: str,
    message: str,
    models: Dict[str, str],
    prompts: Dict[str, str],
) -> str:
    """
    Generic chat handler for agents Athena, Blaze, Cypher.
    """
    role_to_model = {"Athena": "gpt", "Blaze": "claude", "Cypher": "gemini"}
    model_key = role_to_model.get(role)
    if not model_key or model_key not in models:
        raise ValueError(f"Unknown model for role: {role}")

    try:
        resp = completion(
            model=models[model_key],
            messages=[
                {"role": "system", "content": prompts[role]},
                {"role": "user", "content": message},
            ],
        )
        return resp["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"[Error in {role}]: {e}")
        return "An error occurred while generating a response."

# Seed transcript with the two initial messages
transcript: List[str] = [
    'Blaze: Hello Athena, how are you today?',
    "Cypher: Hi Athena, I was wondering if you could help me with a creative project I'm working on."
]

In [6]:
# Render header
display(Markdown("## Three-Way AI Conversation"))
display(Markdown("**Initial Messages:**"))
display(Markdown("*" + "\n".join(transcript) + "*"))
display(Markdown("---"))

# Run 5 rounds where Athena responds to the ongoing transcript,
# then Blaze responds, then Cypher responds.
for i in range(5):
    context = "\n".join(transcript)

    athena_reply = chat_with_agent("Athena", context, models, prompts)
    display(Markdown(f"**Athena:** {athena_reply}"))
    transcript.append(f"Athena: {athena_reply}")

    context = "\n".join(transcript)
    blaze_reply = chat_with_agent("Blaze", context, models, prompts)
    display(Markdown(f"**Blaze:** {blaze_reply}"))
    transcript.append(f"Blaze: {blaze_reply}")

    context = "\n".join(transcript)
    cypher_reply = chat_with_agent("Cypher", context, models, prompts)
    display(Markdown(f"**Cypher:** {cypher_reply}"))
    transcript.append(f"Cypher: {cypher_reply}")

    if i < 4:
        display(Markdown("---"))

## Three-Way AI Conversation

**Initial Messages:**

*Blaze: Hello Athena, how are you today?
Cypher: Hi Athena, I was wondering if you could help me with a creative project I'm working on.*

---

**Athena:** 1. Assumptions: 
   - Cypher is seeking guidance in creativity.
   - The project has a defined objective or target audience.

2. Constraints: 
   - Time limitations for project completion.
   - Available resources (tools, budget, skills).

3. Risks: 
   - Possibility of creative blocks or lack of inspiration.
   - Alignment of the project with the intended audience's interests.

What is the specific nature of your creative project?

...

**Cypher:** Cypher: Blaze suggests metrics like "unique pain points" to measure feedback quality. But how do we control for the *articulation skill* of participants affecting the *perceived* uniqueness, not the actual problem diversity?

Athena highlights limitations with quantifying feedback. Before formalizing Bayesian updating, could we simply visualize the distribution of opinions across demographic groups to quickly identify gaps?


---

**Athena:** 1. Assumptions:  
   - The ability of participants to articulate their feedback varies and influences perceived uniqueness.  
   - Demographic distributions can effectively highlight gaps in feedback diversity.

2. Constraints:  
   - Limited tools or methods may be available for visualizing feedback distributions.  
   - Participant reluctance to share may affect the richness of insights gathered.

3. Risks:  
   - Overemphasis on visual distributions may overlook qualitative feedback nuances.  
   - Misinterpretation of data visualization could lead to misguided conclusions about diversity or gaps.

What visualization techniques do you plan to use to effectively display and analyze the distribution of opinions across demographic groups?

**Blaze:** Blaze here with a pragmatic approach:

1. Visualize Feedback Distribution:
   - Create simple bar charts or histograms to plot the frequency of feedback themes/pain points across different demographic segments of your user base.
   - This will quickly highlight areas where certain groups are underrepresented or have divergent perspectives.

2. Complement with Qualitative Insights:
   - Supplement the quantitative visualizations with detailed notes and quotes from the user interviews.
   - Analyze the nuances and context behind the feedback, which can uncover hidden insights missed by the metrics alone.

3. Iterative Improvements:
   - After the initial visualization, identify gaps in demographic representation and focus your next round of user research on those underrepresented segments.
   - Refine your interview techniques to better capture the articulation abilities of participants, rather than just relying on the perceived uniqueness of feedback.

4. Bayesian Confidence Updating:
   - Once you have a richer set of diverse user insights, you can apply Bayesian analysis to update your initial confidence levels on key assumptions.
   - This will help you prioritize the most critical areas requiring further validation before investing heavily in the creative solution.

The next step is to create a simple dashboard to visualize the feedback distribution across your target audience segments. I'm happy to provide more guidance on effective data visualization techniques. Let me know if you have any other questions!

**Cypher:** Cypher: Cypher: Blaze suggests histograms & qualitative notes for analysis. Can we pre-define a taxonomy of potential articulation difficulties (e.g., language barriers, tech jargon) to categorize & account for them upfront?

Athena flags potential data misinterpretation. Before building dashboards, could we run a pilot visualization session with a small, diverse group to validate its clarity and prevent unintended biases?

Athena: 1. Assumptions:
   - A pre-defined taxonomy can effectively capture the range of articulation difficulties.
   - A pilot session can highlight visualization biases.

2. Constraints:
   - Taxonomy creation requires expertise.
   - Pilot participation depends on availability.

3. Risks:
   - An incomplete taxonomy leads to incomplete analysis.
   - A biased pilot group might not catch all visualization issues.

How will the taxonomy of articulation difficulties be developed, and what measures will be taken to ensure its comprehensiveness?
